# GPT3.5 Function Calling am Beispiel von einer Flugabfrage

In [2]:
# --------------------------------------------------------------
# Import Modules
# --------------------------------------------------------------

import os
import json
import openai
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, ChatMessage



In [3]:

# --------------------------------------------------------------
# Load OpenAI API Token From the .env File
# --------------------------------------------------------------

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")



### Wie funktioniert GPT3.5 überhaupt normal im Code?

In [4]:

# --------------------------------------------------------------
# Ask ChatGPT a Question
# --------------------------------------------------------------

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user","content": "When is the next flight from Düsseldorf to New York?",},
    ],
)

output = completion.choices[0].message.content
print(output)

I'm sorry, but as an AI, I don't have real-time information on flight schedules. It would be best to check with airlines or travel websites for the most up-to-date flight information.


### Ein einfaches Function Calling Beispiel

In [18]:
function_descriptions = [
    {
        "name": "get_flight_info",
        "description": "Get flight information between two locations",
        "parameters": {
            "type": "object",
            "properties": {
                "loc_origin": {
                    "type": "string",
                    "description": "The departure airport abbreviation, e.g. DUS",
                },
                "loc_destination": {
                    "type": "string",
                    "description": "The destination airport abbreviation, e.g. HAM",
                },
            },
            "required": ["loc_origin", "loc_destination"],
        },
    }
]

user_prompt = "When is the next flight from Düsseldorf to New York?"

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": user_prompt}],
    functions=function_descriptions,  # Add function calling
    function_call="auto",
)

output = completion.choices[0].message
print(output) # Was liefert uns GPT hier?  


# -> Wir haben nie erwähnt wie der Flughafen in NewYork heißt

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_flight_info",
    "arguments": "{\n  \"loc_origin\": \"DUS\",\n  \"loc_destination\": \"JFK\"\n}"
  }
}


In [14]:
def get_flight_info(loc_origin, loc_destination):
    """Get flight information between two locations."""

    # Beispiel Ausgabe einer API oder Datenbank
    flight_info = {
        "loc_origin": loc_origin,
        "loc_destination": loc_destination,
        "datetime": str(datetime.now() + timedelta(hours=2)),
        "airline": "KLM",
        "flight": "KL643",
    }

    return json.dumps(flight_info)


# Use the LLM output to manually call the function
# The json.loads function converts the string to a Python object

origin = json.loads(output.function_call.arguments).get("loc_origin")
destination = json.loads(output.function_call.arguments).get("loc_destination")
params = json.loads(output.function_call.arguments)



Schauen wir uns die einzelnen Parameter an

In [15]:
print(origin)
print(destination)
print(params)

DUS
NYC
{'loc_origin': 'DUS', 'loc_destination': 'NYC'}


In [16]:
# Rufe die von uns definierte Funktion mit Parametern auf
chosen_function = eval(output.function_call.name)
flight = chosen_function(**params)

print(flight)


{"loc_origin": "DUS", "loc_destination": "NYC", "datetime": "2023-09-28 16:05:24.853620", "airline": "KLM", "flight": "KL643"}


Diesen Output unserer Funktion können wir nun unserem LLM geben, der dann daraus eine Antwort formuliert

In [17]:

# --------------------------------------------------------------
# Add function result to the prompt for a final answer
# --------------------------------------------------------------

# The key is to add the function output back to the messages with role: function
second_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "function", "name": output.function_call.name, "content": flight},
    ],
    functions=function_descriptions,
)
response = second_completion.choices[0].message.content
print(response)


The next flight from Düsseldorf (DUS) to New York (NYC) is on September 28, 2023, at 16:05. The airline is KLM and the flight number is KL643.
